In [ ]:
from simulation import evaluate, interpolate
import numpy as np
from utils import *
import matplotlib.pyplot as plt
from xfoil import XFoil
from xfoil.model import Airfoil
import os, psutil, gc
process = psutil.Process()

airfoil = np.loadtxt('baseline.dat')
airfoil = interpolate(airfoil, 256, 3)

xf = XFoil()
count = 0
af = np.copy(airfoil)
cl = 0.65
xf.print = 0
xf.airfoil = Airfoil(af[:,0], af[:,1])
xf.Re = 5.8e4
xf.M = 0
xf.max_iter = 2000
a, cd, cm, cp = xf.cl(cl)
perf = cl/cd
cdc = cd

af = setflap(af, theta=-2, pose = 0.7)
xf.airfoil = Airfoil(af[:,0], af[:,1])
xf.Re = 4e5
xf.M = 0
xf.max_iter = 2000
a, cl, cd, cm, cp = xf.aseq(-2, 2, 0.5)
print(perf, cd.min())
del xf

In [ ]:
import numpy as np
from utils import *
points = 'results/airfoilPPO.dat'
airfoil = np.loadtxt(points, skiprows=1)

show_airfoil(airfoil)
airfoil[:,1] = airfoil[:,1] * 0.06 / cal_thickness(airfoil)
af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, lamda=5, check_thickness=False, modify_thickness = True)
name = points.split('/')[1].split('.')[0]+f'_{a}_{b}F'
# np.savetxt(points.split('.')[0]+f'_{a}_{b}F'+'.dat', af, header=points.split('/')[1].split('.')[0]+f'_{a}_{b}F', comments="")
R, a, b, perf, cd, CD_BL

In [ ]:
show_airfoil(af)
detect_intersect(af), cal_thickness(af)

In [ ]:
points = 'bayesoptim/ga_hickhenne.dat'
airfoil = np.loadtxt(points, skiprows=1)
print(cal_thickness(airfoil))
perf, _, cd = evalperf(airfoil, cl = 0.65, Re = 58000)
airfoil = setflap(airfoil, theta=-2)
CD, _ = evalpreset(airfoil, Re=4e5)
R = cd + CD * 5
print('Successful: CL/CD={:.4f}, R: {}, cd: {}, thickness: {}'.format(perf, R, CD, cal_thickness(airfoil)))

In [ ]:
alphas = [-2,-1.5,-1,-0.5,0,0.5,1]
CDs = []
for alpha in alphas:
    xf = XFoil()
    xf.print=0
    xf.airfoil = Airfoil(af[:,0], af[:,1])
    xf.Re=4e5
    xf.max_iter=2000
    _, CD, _, _ = xf.a(alpha)
    CDs.append(CD)
    del xf
CDs

In [ ]:
np.linspace(-1,1,5)

In [ ]:
from utils import *
import re
import os
LAMBDA = 3
dat = re.compile('.dat')
root = 'bayesoptim/'

airfoils = []
for path, dir, files in os.walk(root):
    for file in files:
        if dat.search(file) is not None:
            airfoils.append('{}/{}'.format(path,file))

Rbl = 1
files.sort()
for file in airfoils:
    # points = '{}{}'.format(root, file)
    points = file
    airfoil = np.loadtxt(points, skiprows=1)
    af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, Re2= 400000, lamda = LAMBDA, check_thickness=False, modify_thickness = True)
    if perf > 39.06369801476684 and CD_BL < 0.00485806493088603:
        name = points.split('/')[1].split('.')[0]+f'_{a}_{b}B'
        np.savetxt('BETTER/'+name+'.dat', af, header=name, comments="")
        print(f'R: {R}, angle: {a}, pose: {b}, perf: {perf}, cruise cd: {cd}, launch cd: {CD_BL}, intersection: {detect_intersect(af)}, thickness: {cal_thickness(af)}, thickness pose: {cal_thickness_percent(af)}, tail cross pose: {check_backpoint(af)}')
    os.remove(points)
    print('{} removed'.format(points))

In [2]:
from utils import *
import re
import os
LAMBDA = 3
dat = re.compile('.dat')
dat = re.compile('airfoil.*\.dat')
root = 'BETTER/'

airfoils = []
for path, dir, files in os.walk(root):
    for file in files:
        if dat.search(file) is not None:
            airfoils.append('{}/{}'.format(path,file))

Rbl = 1
files.sort()
for file in airfoils:
    # points = '{}{}'.format(root, file)
    points = file
    airfoil = np.loadtxt(points, skiprows=1)
    af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, Re2= 400000, lamda = LAMBDA, check_thickness=False, modify_thickness = True)
    if perf > 39.06369801476684 and CD_BL < 0.00485806493088603:
        name = points.split('/')[1].split('.')[0]+f'_{a}_{b}B'
        np.savetxt('BETTER/'+name+'.dat', af, header=name, comments="")
        print(f'R: {R}, angle: {a}, pose: {b}, perf: {perf}, cruise cd: {cd}, launch cd: {CD_BL}, intersection: {detect_intersect(af)}, thickness: {cal_thickness(af)}, thickness pose: {cal_thickness_percent(af)}, tail cross pose: {check_backpoint(af)}')
    os.remove(points)
    print('{} removed'.format(points))

BETTER/airfoil007F.dat removed
BETTER/airfoil002_-2.0_0.6F.dat removed
BETTER/airfoil010.dat removed
BETTER/airfoil001_-2.0_0.7.dat removed
BETTER/airfoil010F.dat removed
